In [1]:
# Decorators::

def make_pretty(func):
    def inner():
        print("I got decorated")
        func()
    return inner

def ordinary():
    print("I am ordinary")
    
# make_pretty() is a decorator
ordinary = make_pretty(ordinary)
ordinary()


I got decorated
I am ordinary


In [2]:
# Decorators::

def make_pretty(func):
    def inner():
        print("I got decorated")#FuncStarted
        func()
        #Func Ended
    return inner

# # make_pretty() is a decorator
# ordinary = make_pretty(ordinary)
# ordinary()

@make_pretty
def ordinary():
    print("I am ordinary")

ordinary()


I got decorated
I am ordinary


In [3]:
# Decorating functions with parameters
# parameters of the nested inner() function inside the decorator is same as the parameters of functions it decorates.
# Taking this into account, now we can make general decorators that work with any number of parameter.

def smart_divide(func):
   def inner(a,b):
      print("I am going to divide",a,"and",b)
      if b == 0:
         print("Whoops! cannot divide")
         return

      return func(a,b)
   return inner

@smart_divide
def divide(a,b):
    return a/b

divide(2,5)
divide(2,0)

I am going to divide 2 and 5
I am going to divide 2 and 0
Whoops! cannot divide


In [8]:
# Nested Decorators 

def star(func):
    def inner(*args, **kwargs):
        print("*" * 30)
        func(*args, **kwargs)
        print("*" * 30)
    return inner

def percent(func):
    def inner(*args, **kwargs):
        print("%" * 30)
        func(*args, **kwargs)
        print("%" * 30)
    return inner

# printer = star(percent(printer))
@star
@percent
def printer(msg):
    print(msg)
printer("Hello")

******************************
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Hello
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
******************************
